In [6]:
#import statements

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, losses
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.models import Model

import os

import numpy as np

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.layers import Conv2D, MaxPool2D
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard

from sklearn.metrics import roc_curve, auc

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Conv2DTranspose
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Reshape
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import UpSampling2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras import utils
from tensorflow.keras.layers import Conv2D, MaxPool2D, Conv2DTranspose
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard
from tensorflow.keras import backend as K



# the size of the images in the PCAM dataset
IMAGE_SIZE = 96

In [2]:
#get data from patchCAMELYON dataset (from Constantijn)

def get_pcam_generators(base_dir, train_batch_size=32, val_batch_size=32, class_mode='binary', ae_model=None):

     # dataset parameters
     TRAIN_PATH = os.path.join(base_dir, 'train+val', 'train')
     VALID_PATH = os.path.join(base_dir, 'train+val', 'valid')

     RESCALING_FACTOR = 1./255

     if ae_model is not None:
          ae_model = model_transform(ae_model).model_transform

     # instantiate data generators
     datagen = ImageDataGenerator(rescale=RESCALING_FACTOR, preprocessing_function=ae_model)

     train_gen = datagen.flow_from_directory(TRAIN_PATH,
                                             target_size=(IMAGE_SIZE, IMAGE_SIZE),
                                             batch_size=train_batch_size,
                                             class_mode=class_mode, 
                                             shuffle=False
                                             )


     val_gen = datagen.flow_from_directory(VALID_PATH,
                                             target_size=(IMAGE_SIZE, IMAGE_SIZE),
                                             batch_size=val_batch_size,
                                             class_mode=class_mode,
                                             shuffle=False)
     
     return train_gen, val_gen

class model_transform:
     def __init__(self, ae_model_1):
          self.ae_model_1 = ae_model_1

     def model_transform(self, tensor):
          tensor_adjusted = utils.img_to_array(tensor)
          tensor_adjusted = np.array([tensor_adjusted])
          tensor_adjusted_prediction = self.ae_model_1.predict(tensor_adjusted/255, verbose=None)[0]
          return tensor_adjusted_prediction*255

In [3]:
train_gen, val_gen = get_pcam_generators('C:/Users/20212150/Documents/Year 3/Q3/Project AI mia', class_mode='input')
train_steps = train_gen.n//train_gen.batch_size
val_steps = val_gen.n//val_gen.batch_size

Found 144000 images belonging to 2 classes.
Found 16000 images belonging to 2 classes.


In [7]:
#autoencoder (from Constantijn)

class autoencoder(Sequential):
    def __init__(self):
        super().__init__()
        self.add(Input(shape=(96,96,3)))

    def create_autoencoder_v1(self, layers = 1):
        for i in range(layers):
            self.add(Conv2D(64, (3,3), activation='relu', padding='same'))
            self.add(MaxPool2D((2,2), padding='same'))

            self.add(Conv2D(32, (3,3), activation='relu', padding='same'))
            self.add(MaxPool2D((2,2), padding='same'))

            self.add(Conv2D(32, (3,3), activation='relu', padding='same'))
            self.add(MaxPool2D((2,2), padding='same'))

            self.add(Conv2D(32, (3,3), activation='relu', padding='same'))
            self.add(UpSampling2D((2,2)))

            self.add(Conv2D(32, (3,3), activation='relu', padding='same'))
            self.add(UpSampling2D((2,2)))
            
            self.add(Conv2D(64, (3,3), activation='relu', padding='same'))
            self.add(UpSampling2D((2,2)))

    def create_autoencoder_v2(self, layers = 1):
        for i in range(layers):
            self.add(Conv2D(64, (3,3), activation='relu', padding='same', input_shape=(96,96,3)))
            self.add(MaxPool2D((2,2), padding='same'))
        
            self.add(Conv2D(64, (3,3), activation='relu', padding='same', input_shape=(96,96,3), name='encoder'))
            self.add(MaxPool2D((2,2), padding='same'))

            self.add(Conv2DTranspose(64, (3,3),strides=2 ,activation='relu', padding='same'))
            self.add(Conv2DTranspose(64, (3,3),strides=2 ,activation='relu', padding='same'))


    def compile_model(self):
        self.add(Conv2D(3, (3,3), activation='sigmoid', padding='same'))
        self.compile(optimizer='adam', loss='mean_squared_error')
    
model1 = autoencoder()
model1.create_autoencoder_v2()
model1.compile_model()
model1._name = 'model1'

In [9]:
# save the model and weights
model_name = 'my_first_cnn_model'
model_filepath = model_name + '.json'
weights_filepath = model_name + '_weights.hdf5'

model_json = model1.to_json() # serialize model to JSON
with open(model_filepath, 'w') as json_file:
    json_file.write(model_json) 


# define the model checkpoint and Tensorboard callbacks
checkpoint = ModelCheckpoint(weights_filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
tensorboard = TensorBoard(os.path.join('logs', model_name))
callbacks_list = [checkpoint, tensorboard]


# train the model
train_steps = train_gen.n//train_gen.batch_size
val_steps = val_gen.n//val_gen.batch_size

history = model1.fit(train_gen, steps_per_epoch=train_steps, 
                    validation_data=val_gen,
                    validation_steps=val_steps,
                    epochs=1,
                    shuffle=True,
                    callbacks=callbacks_list)

model1.save('model1.h5')

 278/4500 [>.............................] - ETA: 40:14 - loss: 0.0253

KeyboardInterrupt: 

In [11]:
train_gen_fake, val_gen_fake = get_pcam_generators('C:/Users/20212150/Documents/Year 3/Q3/Project AI mia', class_mode="binary", ae_model=model1)

Found 144000 images belonging to 2 classes.
Found 16000 images belonging to 2 classes.


In [1]:
#combine train_gen with train_gen_fake (used in next cell)


def combine_gen(*gens):
    while True:
        for g in gens:
            yield next(g)

In [21]:
# FCNN trained with normal dataset and augmented dataset

train_gen, val_gen = get_pcam_generators('C:/Users/20212150/Documents/Year 3/Q3/Project AI mia', class_mode='binary')


def get_model(kernel_size=(3,3), pool_size=(4,4), first_filters=32, second_filters=64):

     # build the model
     model = Sequential()

     model.add(Conv2D(first_filters, kernel_size, activation = 'relu', padding = 'same', input_shape = (IMAGE_SIZE, IMAGE_SIZE, 3)))
     model.add(MaxPool2D(pool_size = pool_size))

     model.add(Conv2D(second_filters, kernel_size, activation = 'relu', padding = 'same'))
     model.add(MaxPool2D(pool_size = pool_size))

     # layers replacing the dense layers (explained in pdf)
     model.add(Conv2D(second_filters, (6,6), activation = 'relu', padding = 'valid'))
     model.add(Conv2D(1, (1,1), activation = 'sigmoid', padding = 'same'))
     model.add(GlobalAveragePooling2D())

     # compile the model
     model.compile(SGD(learning_rate=0.01, momentum=0.95), loss = 'binary_crossentropy', metrics=['accuracy'])

     return model

model = get_model()
# save the model and weights
model_name = 'FCNN+autoencoder'
model_filepath = model_name + '.json'
weights_filepath = model_name + '_weights.hdf5'

model_json = model.to_json() # serialize model to JSON
with open(model_filepath, 'w') as json_file:
    json_file.write(model_json)

# define the model checkpoint and Tensorboard callbacks
checkpoint = ModelCheckpoint(weights_filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
tensorboard = TensorBoard(os.path.join('logs', model_name))
callbacks_list = [checkpoint, tensorboard]

# train the model
train_steps = train_gen.n//train_gen.batch_size
val_steps = val_gen.n//val_gen.batch_size

history = model.fit(combine_gen(train_gen, train_gen_fake), steps_per_epoch=len(train_gen)+len(train_gen_fake),
                    validation_data=combine_gen(val_gen, val_gen_fake),
                    validation_steps=len(train_gen)+len(train_gen_fake),
                    epochs=3,
                    callbacks=callbacks_list)


Found 144000 images belonging to 2 classes.
Found 16000 images belonging to 2 classes.
Epoch 1/3
 385/9000 [>.............................] - ETA: 2:06:16 - loss: 0.0079 - accuracy: 0.9974

KeyboardInterrupt: 